## Preparing the dataset
- load the data
- rename the columns
- split the dataset into train and test (test will be used only at the very final step)
- separate the independent variables from the target variables

In [1]:
import pandas as pd
import numpy as np

# load the data
leaf = pd.read_csv("leaf.csv" , header = None)

# rename the columns
names = np.array(["Class", "Specimen", "Eccentricity", "Aspect Ratio", "Elongation",
                  "Solidity", "Stochastic Convexity", "Isoperimetric Factor",
                  "Maximal Indentation Depth ","Lobedness","Average Intensity",
                  "Average Contrast","Smoothness","Third moment","Uniformity","Entropy"])
leaf.columns = names

data = leaf.loc[:, leaf.columns != 'Specimen']

"""
# split the dataset into train and test
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, random_state = 5, test_size = 0.2, stratify = data['Class'])
train.index = np.linspace(0, len(train)-1, len(train), dtype = 'int')
test.index = np.linspace(0, len(test)-1, len(test), dtype = 'int')


# separate the independent variables from the target variables
X = train.loc[:, train.columns != 'Class']
y = train['Class']
"""

X = data.loc[:, data.columns != 'Class']
y = data['Class']

FileNotFoundError: [Errno 2] No such file or directory: 'leaf.csv'

## Building some useful tools
- declaring two cross-validation loop: 
    - outer stratified cv-loop
    - inner non-stratified cv-loop
- set the scoring we'd like to compute (this tuple will be used inside 'do_cross_validation')
- definition of do_cross_validation: it evaluate metric(s) by cross-validation and also record fit/score times, then print result(s).

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, KFold, LeaveOneOut, cross_validate

# declaring two cross-validation loop
inner_cv = StratifiedKFold(n_splits = 5) # random_state = 10)
outer_cv = StratifiedKFold(n_splits = 5, shuffle = True)

# set the scoring
scoring = ('f1_weighted', 'accuracy', 'f1_macro', 'f1_micro', 'roc_auc_ovo_weighted', 'roc_auc_ovr_weighted')

# definition of do_cross_validation
def do_cross_validation(clf,X=X, y=y, print_model=True, scoring = scoring):
    cv = cross_validate(clf, X, y, scoring=scoring, cv= outer_cv, return_train_score=False)

    for i in range(len(scoring)):
        scores = ' + '.join(f'{s:.2f}' for s in cv['test_' + scoring[i]])
        mean_ = cv['test_' + scoring[i]].mean()
        msg = f'Cross-validated {scoring[i]}: ({scores}) / {outer_cv.n_splits} = {mean_:.2f}'
        if print_model:
            msg = f'{clf}:\n\t{msg}\n'
        print(msg)

In [ ]:
from sklearn.model_selection import GridSearchCV

## Train RandomForest
- train a RandomForest by a two nested CV loop. 
    - in the outer loop we estimate the score indexes using 'do_cross_validation'
    - in the inner loop we to GridSearch

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# random forest inner loop
param_grid={'n_estimators': [50, 100, 500], 'criterion': ["gini", "entropy"]}
clf_grid = GridSearchCV(RandomForestClassifier(random_state=0), param_grid=param_grid, cv = inner_cv, n_jobs=4)
# random forest outer loop
do_cross_validation(clf_grid)

## Standardization

before moving to SVM and kNN, we need to standardize our independent variable. Since we are using cross validation also to assess our final performance metric we need to ensure that standardization is performed in each fold and not on the whole dataset.\
In order to do this we need to use the sklearn `pipeline` in conjunction with nested cross validation as before.\
**Performing standardization on the whole set would be wrong as we'd have information leakage from the data we use to perform our cross validation**\

When you use parameter_grid remember that the parameters must have the name of the classifier

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

## Train SVM with Gaussian Kernel 
- train a SVM by a two nested CV loop. 
    - in the outer loop we estimate the score indexes using 'do_cross_validation'
    - in the inner loop we to GridSearch

Hyper-paramters to consider in the grid search:
 - `gamma`: use a set of float values, sklearn documentation suggests using exponentially spaced ones (and so do other tutorials).
 - `C`: use a set of float values, sklearn documentation suggests using exponentially spaced ones (and so do other tutorials).

In [ ]:
from sklearn.svm import SVC
pipeline_SVC = Pipeline([('scaler', StandardScaler()),
                        ('classifier', SVC(kernel='rbf', probability = True))]) #If we choose some scoring without probability set probability to FALSE
param_grid = {'classifier__C': np.logspace(-2, 5, 8), 'classifier__gamma' : np.logspace(-4, 3, 8)}
grid_search = GridSearchCV(pipeline_SVC, param_grid=param_grid, cv = inner_cv, n_jobs=4)

In [ ]:
do_cross_validation(grid_search)

## Train kNN
- train a kNN by a two nested CV loop. 
    - in the outer loop we estimate the score indexes using 'do_cross_validation'
    - in the inner loop we to GridSearch

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
pipeline_KNN = Pipeline([('scaler', StandardScaler()),
                        ('classifier', KNeighborsClassifier())])

param_grid = {'classifier__n_neighbors': (2, 5, 10, 15, 20), 'classifier__p':[1, 2]}
grid_search = GridSearchCV(pipeline_KNN, param_grid=param_grid, cv = inner_cv)

In [ ]:
do_cross_validation(grid_search)

## Next step (asap)

- MICHELE: effectiveness indexes (scegliere quale usare) 
- ELENA: cross validation: inner outer (probabilmente sara: stratified e il numero dei fold è tale che non abbiamo fold vuoti rispetto a una classe)
- SAMUELE: vedere di sistemare la stadardizzazione 

- scegliere gli iperparametri di:
        - ELENA: albero (aggiungere albero singolo all'inizio) + RF (max_features + quanti alberi?) + kNN
        - MICHELE: SVM (linear)
        - SAMUELE: SVM (gauss)


- analisi variabili ???